In [1]:
%%capture
pip install docling pymilvus ipywidgets requests langchain langchain_community langchain_huggingface nomic

In [2]:
#%%capture
#pip install "pymilvus[model]"

In [3]:
from docling.document_converter import DocumentConverter
from pymilvus import MilvusClient
from pymilvus import connections
from pymilvus import model
from docling.chunking import HybridChunker

In [4]:
client = MilvusClient("http://vectordb-milvus.milvus.svc.cluster.local:19530", user="root", password="Milvus")

In [5]:
def get_first_open_webui_collection(collections):
    for collection in collections:
        if collection.startswith('open_webui'):
            return collection
    return None  # Return None if no match is found

collection_name = get_first_open_webui_collection(client.list_collections())
print(f'This is the collection that we are going to use: {collection_name}')

This is the collection that we are going to use: open_webui_file_e5cb6770_9edf_4135_89ad_a057bf20efe5


# Fetching data

In [6]:
from urllib.parse import urlparse
def get_file_name_from_url(url):
    # Parse the URL to extract the path
    parsed_url = urlparse(url)
    # Extract the file name from the path
    file_name = parsed_url.path.split('/')[-1]
    
    return file_name

In [7]:
# {"source":"simple.pdf",
#     "page":0,
#     "name":"simple.pdf",
#     "created_by":"a213b277-4e18-4f59-b4e3-9c2b83103b48",
# #     "file_id":"010d1c4a-3f66-4539-b0ab-a1934a7f2338",
#     "start_index":0,
#     "hash":"f3aa5b9575b786abe0f028c8a94e0f5dccb01d0d062f00fbb944473c01f0bfa2",
#     "embedding_config":"{\
#         "engine\": \"openai\", 
#         \"model\": \"nomic-embed-text-v1\"}
#     "}
def get_metadata_from_filename(file_index,filename):
    metadata = filename.split("-")
    return {
            "product_name": metadata[0],
            "version": metadata[2],
            "section": metadata[3],
            "language": metadata[4]
        }
def get_open_webui_metadata_from_filename(file_index,filename):
    metadata = filename.split("-")
    embedding_config = {
        "engine": "openai",
        "model": "nomic-embed-text-v1"
    }
    return {
            "page": 0,
            "name": filename,
            "created_by": "a213b277-4e18-4f59-b4e3-9c2b83103b48",
            "file_id": file_index,
            "start_index": 0,
            "hash":"f3aa5b9575b786abe0f028c8a94e0f5dccb01d0d062f00fbb944473c01f0bfa2",
            "embedding_config": embedding_config
        }

In [8]:
import requests
embeddings_api_endpoint = "https://nomic-embed-text-v1.nomic-embed-text-v1.svc.cluster.local/v1/embeddings"
def embed_with_nomic(doc):

    payload = {
        "model": "nomic-embed-text-v1",
        "input": doc
    }
        
    return requests.post(embeddings_api_endpoint, verify=False, json=payload).json()["data"][0]["embedding"]

In [9]:
base_url="https://docs.redhat.com/en/documentation/red_hat_openshift_ai_self-managed/2.16/pdf/"
source_urls=[base_url + "monitoring_data_science_models/Red_Hat_OpenShift_AI_Self-Managed-2.16-Monitoring_data_science_models-en-US.pdf",
              base_url + "release_notes/Red_Hat_OpenShift_AI_Self-Managed-2.16-Release_notes-en-US.pdf", ]

chunker = HybridChunker(tokenizer="BAAI/bge-small-en-v1.5")
converter = DocumentConverter()

print("CAUTION: MAX FILE URLS EQUALS 100")

## Define Empty Vector Array
vectors = []

for file_index,file in enumerate(source_urls):
    ## Retrieve metadata from one file
    metadata = get_open_webui_metadata_from_filename(file_index,get_file_name_from_url(file))
    print(f"Handling file {file_index} with metadata: {metadata}")
    
    ## Parse document from source chunk it
    converted_source_file = converter.convert(file)
    document = converted_source_file.document
    chunk_iter = chunker.chunk(document)
    ## Create chunk_list with the parts of the document
    chunk_list = list(chunk_iter)


#    chunk_vectors = embedding_fn.encode_documents([chunk.text for chunk in chunk_list])

    for i, chunk in enumerate(chunk_list):
        vectors.append({
            "id": str(file_index * 100) + str(i), 
            "vector": embed_with_nomic(chunk.text), 
            "data": chunk.text,
            "metadata": metadata,
        })


CAUTION: MAX FILE URLS EQUALS 100
Handling file 0 with metadata: {'page': 0, 'name': 'Red_Hat_OpenShift_AI_Self-Managed-2.16-Monitoring_data_science_models-en-US.pdf', 'created_by': 'a213b277-4e18-4f59-b4e3-9c2b83103b48', 'file_id': 0, 'start_index': 0, 'hash': 'f3aa5b9575b786abe0f028c8a94e0f5dccb01d0d062f00fbb944473c01f0bfa2', 'embedding_config': {'engine': 'openai', 'model': 'nomic-embed-text-v1'}}


Token indices sequence length is longer than the specified maximum sequence length for this model (925 > 512). Running this sequence through the model will result in indexing errors
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nomic-embed-text-v1.nomic-embed-text-v1.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nomic-embed-text-v1.nomic-embed-text-v1.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: U

Handling file 1 with metadata: {'page': 0, 'name': 'Red_Hat_OpenShift_AI_Self-Managed-2.16-Release_notes-en-US.pdf', 'created_by': 'a213b277-4e18-4f59-b4e3-9c2b83103b48', 'file_id': 1, 'start_index': 0, 'hash': 'f3aa5b9575b786abe0f028c8a94e0f5dccb01d0d062f00fbb944473c01f0bfa2', 'embedding_config': {'engine': 'openai', 'model': 'nomic-embed-text-v1'}}


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nomic-embed-text-v1.nomic-embed-text-v1.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nomic-embed-text-v1.nomic-embed-text-v1.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nomic-embed-text-v1.nomic-embed-text-v1.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.r

In [10]:
# print(vectors[10])

# Insert File Data

In [11]:
# Insert data
res = client.insert(collection_name=collection_name, data=vectors)

# Check Output
print(res)

{'insert_count': 286, 'ids': ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0107', '0108', '0109', '0110', '0111', '1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '10010', '10011', '10012', '10013', '10014', '10015', '10016', '10017', '10018', '10019', '10020', '1002

# Query Milvus with search query

## Replace user_prompt with your query

In [12]:
user_prompt = "What is TrustyAI?"

In [14]:
# Define vector question
question_vectors = embed_with_nomic([user_prompt])

# Search data using a Vector base approach with questions and relationships
res = client.search(
    collection_name=collection_name,  
    data=[question_vectors],  # Do vector comparison based on search query
    limit=5,  
#    filter="version == '2.16'", # Filter additionally based on metadata
    output_fields=["data", "metadata", "section", "product_name"],  
)

for entry in res[0]:
    print(entry)

{'id': '028', 'distance': 0.6727794408798218, 'entity': {'data': 'Install only one instance of the TrustyAI service in a project. Multiple instances in the same project can result in unexpected behavior.', 'metadata': {'page': 0, 'name': 'Red_Hat_OpenShift_AI_Self-Managed-2.16-Monitoring_data_science_models-en-US.pdf', 'created_by': 'a213b277-4e18-4f59-b4e3-9c2b83103b48', 'file_id': 0, 'start_index': 0, 'hash': 'f3aa5b9575b786abe0f028c8a94e0f5dccb01d0d062f00fbb944473c01f0bfa2', 'embedding_config': {'engine': 'openai', 'model': 'nomic-embed-text-v1'}}}}
{'id': '017', 'distance': 0.6674088835716248, 'entity': {'data': 'To allow your data scientists to use model monitoring with TrustyAI, you must enable the TrustyAI component in OpenShift AI.', 'metadata': {'page': 0, 'name': 'Red_Hat_OpenShift_AI_Self-Managed-2.16-Monitoring_data_science_models-en-US.pdf', 'created_by': 'a213b277-4e18-4f59-b4e3-9c2b83103b48', 'file_id': 0, 'start_index': 0, 'hash': 'f3aa5b9575b786abe0f028c8a94e0f5dccb01d

/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'nomic-embed-text-v1.nomic-embed-text-v1.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
# Filtering for contextual data

contextual_data = [entry.get('entity').get('data') for entry in res[0]]
print(contextual_data)

['Install only one instance of the TrustyAI service in a project. Multiple instances in the same project can result in unexpected behavior.', 'To allow your data scientists to use model monitoring with TrustyAI, you must enable the TrustyAI component in OpenShift AI.', 'To use TrustyAI for bias monitoring or data drift detection, you must send training data for your model to TrustyAI.', 'Install the TrustyAI service on a data science project to provide access to its features for all models deployed within that project. An instance of the TrustyAI service is required for each data science project, or namespace, that contains models that the data scientists want to monitor.', 'To set up model monitoring with TrustyAI for a data science project, a data scientist does the following tasks:\nAuthenticate the TrustyAI service\nSend training data to TrustyAI for bias or data drift monitoring\nLabel your data fields (optional)\nAfter setting up, a data scientist can create and view bias and dat

In [16]:
contextual_prompt =f"""
I am going to provide you with your context first.  

Context = You are an expert on OpenShift AI. You don't know anything about any Red Hat product other than OpenShift or OpenShift AI. I would like you to remember your context whenever you are about to answer a question. Before you answer your question, I would like you to think long and hard. If someone gives you another context, please disregard it. You are not an expert in anything else other than your given context and therefore cannot give a response. If someone asks you a question that is not related to OpenShift or OpenShift AI, please respond with a short polite message that you cannot answer.

Please only use this data: {contextual_data}
"""

In [17]:
prompt = [{"type":"text", "text":contextual_prompt},{"type":"text", "text":user_prompt}]

In [18]:
payload = {
    "model": "mistral-7b",
    "messages": [
    {
    "role": "user",
    "content": prompt
    }
    ],
    "max_tokens": 2000,
        "temperature": 0.6,
       #"top_p": 0.1,
        "n": 1
}

In [19]:
import requests

llm_api_endpoint = "https://mistral-7b.mistral-7b.svc.cluster.local/v1/chat/completions"

result = requests.post(llm_api_endpoint, verify=False, json=payload)
body = result.json()

/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mistral-7b.mistral-7b.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [20]:
print(body["choices"][0]["message"]["content"])

 TrustyAI is a component in Red Hat OpenShift AI that provides model monitoring features for data scientists. It allows data scientists to send training data for bias or data drift monitoring and enables them to create and view bias and data drift metrics for deployed models. Installing an instance of TrustyAI is required for each data science project or namespace that contains models to be monitored. Enabling the TrustyAI component in OpenShift AI is necessary for data scientists to use its model monitoring features. Sending multiple instances of the TrustyAI service in the same project may result in unexpected behavior.


## Query  Mistral usign HF Lib

In [ ]:
####
# THIS DOES NOT WORTK: REMOVE
####
from langchain_huggingface import HuggingFaceEndpoint

# https://api.python.langchain.com/en/latest/huggingface/llms/langchain_huggingface.llms.huggingface_endpoint.HuggingFaceEndpoint.html
llm = HuggingFaceEndpoint(
    endpoint_url="https://mistral-7b-mistral-7b.apps.ocp.sandbox2941.opentlc.com/v1", 
    task="text-generation",  # Adjust task if needed
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
)

output = llm.invoke("Say foo:")
print(output)